<a href="https://colab.research.google.com/github/bokutachi256/gisday2024/blob/main/GISDAY2024_%E8%A1%8C%E5%8B%95%E3%83%8F%E3%82%9A%E3%82%BF%E3%83%BC%E3%83%B3%E3%81%AE%E5%88%86%E6%9E%90%E5%85%A5%E9%96%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# scikit-mobilityを用いた行動パターンの分析入門

* 東京都立大学 都市環境学部 地理環境学科 中山大地
* GIS Day in 東京 2024 Eコース
* 2024年12月14日 東京都立大学 南大沢キャンパス
* このテキストのURL [https://github.com/bokutachi256/gisday2024](https://github.com/bokutachi256/gisday2024)
* Google ColaboratoryのURL [https://colab.research.google.com/](https://colab.research.google.com/)

## 使い方

### 動作環境について

1. Googleドライブに`gisday2024`フォルダを作成する（フォルダ名は「すべて小文字」にすること）．
2. 共有フォルダ（[https://drive.google.com/drive/folders/1OKj-JS8O2NR2qPlvLV4oiTNdrMEV8Jpn?usp=sharing](https://drive.google.com/drive/folders/1OKj-JS8O2NR2qPlvLV4oiTNdrMEV8Jpn?usp=sharing)）から以下のデータをダウンロードする．
   * `gps_logdata.zip`: GPX型式のGPSログデータを圧縮したもの
   * `r2ka.fgb`: FlatGeoBuf型式の熊本県と宮崎県の小地域ポリゴン（e-Statより入手）
   * `gisday2024_agent_vars.fgb`: FlatGeoBuf型式のマルチエージェントシミュレーションの計算結果（熊本県阿蘇市内牧地区）
3. Google Driveに作成した`gisday2024`フォルダに上記3点のファイルをアップロードする．


#### 使用しているライブラリ

使用している主なライブラリの一覧．

* gpxpy: GPSのログデータ（GPX型式）を扱うためのライブラリ
* scikit-mobility: モビリティデータを扱うライブラリ
* movingpandas: 移動データを扱うライブラリ
* folium: 地図（WEBマップ）を扱うためのライブラリ
* datetime: 時刻を扱うためのライブラリ
* zoneinfo: タイムゾーンを扱うためのライブラリ
* pandas: データフレームを扱うためのライブラリ
* geopandas: 地理情報を扱うためのライブラリ
* pyproj: PROJ（地図投影と座標系を扱うためのライブラリ）へのPythonインターフェイス
* glob: ファイルパス名解決のためのライブラリ
* shapely: ジオメトリを扱うためのライブラリ
* numpy: 行列を扱うためのライブラリ

## 実行の準備

### ライブラリのインストールとインポート

#### Google Colabへのライブラリインストール

Google Colaboratoryは基本的なライブラリはインストール済みですが，標準ではインストールされていないライブラリをインストールする必要があります．
インストールするとランタイムの再起動を求められる事があるので，指示に従ってランタイムを再起動してください．
再起動後は「ライブラリのインポート」以降を実行してください．

In [ ]:
import sys

# Google Colaboratoryを使用している場合は以下の必須ライブラリをインストールする
# gpxpy: GPXフォーマットを扱うライブラリ
# scikit-mobility: モビリティデータを扱うライブラリ
# movingpandas: 移動データを扱うライブラリ
# folium: 地図を表示するライブラリ

if 'google.colab' in sys.modules:
  %pip install gpxpy
  %pip install scikit-mobility
  %pip install movingpandas
  %pip install folium
  %pip install mapclassify
  %pip install -U geopandas
  %pip install stonesoup


#### ライブラリのインポート

In [ ]:
# 必要なライブラリをインポート
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import sys
import gpxpy
import gpxpy.gpx
import folium
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import skmob
from skmob import TrajDataFrame
from skmob.preprocessing import detection, clustering, compression
from skmob.preprocessing import filtering
from skmob.tessellation import tilers

from shapely.geometry import Point, LineString
from pyproj import CRS
import glob
import zipfile

import numpy as np

### データフォルダのマウント

Google ColaboratoryからGoogle Driveにアップロードしたデータにアクセスするため，
データフォルダのマウントを行います．
途中でGoogle ColaboratoryからGoogle Driveへのアクセス権付与を求められますので，すべての権限を与えてください．


In [ ]:
if 'google.colab' in sys.modules:
  # google.coalb: GoogleDrive内のファイルにアクセスするために必要
  from google.colab import drive
  drive.mount('/content/drive')
  # GoogleDrive内の"マイドライブ/gisday2024/"フォルダにアクセスできるような設定を行う．
  # フォルダ名の最後に`/`を必ず追加すること．
  base_dir = "/content/drive/MyDrive/gisday2024/"
else:
  base_dir = "./data/"

%matplotlib inline

# GPXデータの分析

熊本県阿蘇地方近辺の31日分のGPSログデータ（学外実習時に取得したものです）を用いてデータの分析を行います．
ここではGPSログデータから立ち寄り地（採水ポイントや現地調査のポイント）を抽出し，
立ち寄り地間のフローを作成します．

手順は以下になります．

1. GPXデータの読み込みとTrajDataFrameへの変換
2. フィルタを用いたTrajDataFrameのクリーニングおよび縮約
3. 立ち寄り地点の抽出
4. 立ち寄り地点間のOD作成
5. ODからフローの作成

## GPSからTrajDataFrameを作る

### GPXデータの読み込み

zip型式に圧縮されているGPXファイルを読み込み，DataFrame型式に変換した後にTrajDataFrameを作成します．
TrajDataFrameはscikit-mobikityの基本的なデータ形式で，
TrajDataFrameに対して様々なフィルタをかけたりデータの縮約をします．
また，立ち寄り地の検出もTrajDataFrameに対して行います．

GPXデータの読み込みはgpxpyライブラリを使用します．
GPXのトラックを構成するセグメントからポイントを抽出し，緯度経度の座標や時刻を取得します．
取得した座標などの情報からpandasのDataFrameを作成し，それをTrajDataFrameに変換します．

In [ ]:
# タイムゾーンの設定
dt_tz = ZoneInfo('Asia/Tokyo')
# 日時文字列形式
dt_fmt = '%Y-%m-%d %H:%M:%S'
# 配列の初期化
datetime = []
lat = []
lon = []
alt = []
id = []
uid = 0
df = pd.DataFrame()

# ZIP圧縮されたGPXファイルの読み込み
inzipfile =  'gps_logdata.zip'
with zipfile.ZipFile(base_dir + inzipfile) as zf:
    infiles = zf.namelist()
    for infile in infiles:
        gpx_file = zf.open(infile, 'r')
        # GPXファイルのパース
        gpx = gpxpy.parse(gpx_file)
        # GPXデータの読み込み
        for track in gpx.tracks:
            for segment in track.segments:
                # ポイントデータリストの読み込み
                points = segment.points
                # ポイントデータの読み込み
                for i in range(len(points)):
                    # データ抽出
                    datetime.append(points[i].time.astimezone(dt_tz).strftime(dt_fmt))
                    lat.append(points[i].latitude)
                    lon.append(points[i].longitude)
                    alt.append(points[i].elevation)
                    id.append(uid)
        uid += 1

df['lat'] = lat
df['lon'] = lon
df['alt'] = alt
df['datetime'] = datetime
df['id'] = id


### TrajDataFrameの作成

GPXデータから作成したDataFrameをTrajDataFrameに変換します．
TrajDataFrameには個人を識別する`user_id`がありますが，
今回は31日分のGPSデータのそれぞれに異なる`user_id`を付与しています．

In [ ]:
# gpxをTrajDataFrameに変換
tdf = skmob.TrajDataFrame(df, latitude='lat', longitude = 'lon', datetime='datetime',
                          user_id='id', crs={'init': 'epsg:4326'})

GPXデータからTrajDataFrameができましたので，データ数の確認をしてみます．

In [ ]:
print(f'オリジナル: {len(tdf)}')

全部で128,802個のポイントがありました．

## TrajDataFrameのクリーニングと各種前処理

GPSの位置情報にはノイズが含まれていることがあるため，それらを取り除く必要があります．
そのためにデータのクリーニングを行います．
まずは確認のためTrajDataFrameをプロットしてみます．

TrajDataFrameのプロットは`plot_trajectory`メソッドを使います．

In [ ]:
# TrajDataFrameのプロット
tdf.plot_trajectory(zoom=12, weight=3, opacity=0.9, max_users=100, max_points=None,
                    start_end_markers=True)


各Trajectoryの始点と終点にマーカーが表示されています．クリックすると日付時刻と座標がポップアップします．

### 速度の計算

GPSデータのノイズとして異常に速い速度が記録される事があります．
確認のため経路ごとの速度を求めて地図表示してみましょう．

scikit-mobilityには経路の速度を計算する方法がないため，
TrajDataFrameをMovingPandsのTrajectoryCollectionに変換して速度を求めます．
まずはTrajDatFrameをGeoPandasのGeoDataFrameに変換し，
そこからTrajctoryCollectionに変換します．
その後，TrajectoryCollectionに対して速度を求めます．

In [ ]:
# TrajDataFrameをMovingPandasのTrajectoryCollectionに変換する

# TrajDataFrameをGeoDataFrameに変換する
gdf = tdf.to_geodataframe()
# GeoDataFrameをTrajectoryCollectionに変換する
collection = mpd.TrajectoryCollection(gdf, 'uid', t='datetime')

# TrajectoryCollectionに対して速度を計算する
collection.add_speed(overwrite=True, units=("km", "h"))


TrajectoryCollectionに対して`add_speed`メソッドを使って速度を求めます．
求まった速度は`speed`に格納されます．

速度が求まったら表示してみましょう．
速度を求めたTrajectoryCollectionはGeoPandasのGeoDataFrameオブジェクトなので，
`plot`メソッドや`explore`メソッドが使えます．

`plot`メソッドを使って`speed`で色分けして表示します．

In [ ]:
# すべてのTrajectoryのspeedをプロットする
# やけにスピードが速いものがある
collection.plot(column="speed", cmap='rainbow', legend=True)

どうやら時速800kmを越えるセグメントがあるようです．
これは明らかにノイズなのでフィルタリングする必要があります．

### データのクリーニング


主なフィルタリングには以下の2種類があります．
* 速度によるフィルタリング
* 短小ループの削除

またデータを縮約するための経路圧縮も必要になります．

まずは速度フィルタリングを行います．
ここでは時速100km以上のセグメントを削除します．
速度フィルタリングは`filtering.filter`を使います．

対象となるTrajDataFrameは`tdf`ですので，`filtering.filter`の第一引数に`tdf`を指定します．
`max_speed_km`で指定する値以上のセグメントを削除します．
フィルタリング後のデータは`stdf`とします．

In [ ]:
#速度によるフィルタリング
# TrajDataFrameのうち，速度が100km/hを超えるデータを除外
stdf = filtering.filter(tdf, max_speed_kmh=100)


速度フィルタリングが終わったらデータ数の確認をしてみましょう．

In [ ]:
print(f'オリジナル: {len(tdf)}')
print(f'速度フィルタリング後: {len(stdf)}')

オリジナルデータに比べてフィルタリング後のデータは小さくなっていることがわかります．


次に速度フィルタを通したTrajDataFrameを表示して確認してみます．
`plot_trajectory`メソッドを使います．

In [ ]:
stdf.plot_trajectory(zoom=12, weight=3, opacity=0.9, max_users=100, max_points=None)


#### 速度をフィルタリングしたTrajectoryDataFrameに対して速度を計算する

プロットではフィルタリングの効果がわからないので，
速度フィルタリング済みのデータに対して再度速度を計算し，
それをプロットして確認します．

速度計算のためには一旦MovingPandasのTrajectoryCollectionに変換する必要があります．


In [ ]:
# TrajDataFrameをGeoDataFrame経由でMovingPandasのTrajectoryCollectionに変換する
gdf = stdf.to_geodataframe()
collection = mpd.TrajectoryCollection(gdf, 'uid', t='datetime')
# TrajectoryCollectionの速度を計算する
collection.add_speed(overwrite=True, units=("km", "h"))
# TrajectoryCollectionを日付でソートする
collection.trajectories.sort(key=lambda x: x.get_start_time())

速度のプロットを表示してみましょう．

In [ ]:
# すべてのTrajectoryのspeedをプロットする
collection.plot(column="speed", cmap='rainbow', legend=True)

最高速度が時速100kmになっているので，速度フィルタリングはうまくいっているようです．

次に`explore`メソッドを使ってインタラクティブマップに速度を日付ごとに表示してみます．

#### インタラクティブマップに経路の速度を表示する

In [ ]:
mapcenter = [32.92, 131.1]

m = folium.Map(
    location=mapcenter,
    zoom_start=11
)

# 地理院タイル（淡色地図）を追加
# '全国最新写真（シームレス）'も面白いかも
# https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg
tile_layer = folium.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='地理院地図'
)
tile_layer.add_to(m)

# カスタムCSSを追加してタイルレイヤーをモノクロ化
m.get_root().html.add_child(folium.Element("""
<style>
    .leaflet-tile {
        filter: grayscale(100%);
    }
</style>
"""))

# レイヤーコントロールに日付を表示するための日付フォーマット
dt_fmt = '%Y-%m-%d'
# TrajectoryCollectionの各Trajectoryをプロット
for trajectory in collection.trajectories:
    # Trajectoryの開始時刻を取得
    start_time = trajectory.get_start_time().strftime(dt_fmt)
    # Trajectoryを表示
    trajectory.explore(
        m=m,
        column="speed",
        cmap='rainbow',
        vmin=0, vmax=100,
        legend=False,
        style_kwds={'weight': 5, 'opacity': 0.7},
        name=f'{start_time}'
    )

# レイヤーコントロールを追加
folium.LayerControl().add_to(m)
display(m)

`explore`メソッドを使っているので，右上のレイヤーコントロールメニューで表示のオンオフを変えることができます．
また，マウスオーバーすることによりセグメントの情報をポップアップすることもできます．

南側の速度が速いセグメントは，部分開通している九州中央自動車道を通行しているときのものです．

#### ループの削除

GPSデータのノイズには短小ループが作成される事があります．
短小ループの削除は速度によるフィルタリングと同時に行います．
これも実行してみます．

In [ ]:
lstdf = filtering.filter(tdf, max_speed_kmh=100, include_loops=True)

これでフィルタリングによるデータクリーニングは完了です．
クリーニング前後のデータ数を見て，効果を見てみましょう．

In [ ]:
print(f'オリジナル: {len(tdf)}')
print(f'速度フィルタリング後: {len(stdf)}')
print(f'ループを含む速度フィルタリング後: {len(lstdf)}')


データ数が少なくなっているのでクリーニングはできていますが，思ったよりもデータは減っていないようです．
マップ表示での確認もしてみます．

In [ ]:
lstdf.plot_trajectory(zoom=12, weight=3, opacity=0.9, max_users=100, max_points=None)


#### データの圧縮

このままではデータ量が多くて扱いが不便なので，`compression.comress`を使って経路データの圧縮を行います．
`spatial_radius_km`パラメーターで削減する範囲をしています．ここでは50mとしました．

In [ ]:
clstdf = compression.compress(lstdf, spatial_radius_km=0.05)

経路圧縮の効果を見てみましょう．

In [ ]:
print(f'オリジナル: {len(tdf)}')
print(f'速度フィルタリング後: {len(stdf)}')
print(f'ループを含む速度フィルタリング後: {len(lstdf)}')
print(f'圧縮後: {len(clstdf)}')

50m範囲での経路圧縮をすることにより，元データの3分の1ほどのサイズになりました．

オリジナルデータと圧縮データを地図表示して効果を見てみます．
すべてのTrajetoryを表示するのは難しいので，最初のTrajectory（`uid`が0のTrajectory ）のみを表示します．

オリジナルデータは赤で，フィルタリングして圧縮したデータは青で表示されます．

In [ ]:
# オリジナルデータを赤で描画
map_f = clstdf[clstdf['uid']==10].plot_trajectory(zoom=12, weight=3, opacity=0.9,
  max_users=100, max_points=None, hex_color='#FF0000')
# 圧縮後のデータを青で描画
tdf[tdf['uid']==10].plot_trajectory(zoom=12, weight=3, opacity=0.5,
  max_users=100, max_points=None, hex_color='#0000FF', map_f=map_f)

#### データの処理過程の確認

オリジナルのTrajectoryに対して様々なクリーニングと圧縮をしましたが，
あるTrajectoryがどのような処理を施されたのか確認したいことがあります．

TrajDataFrameの`parameters`メソッドで適用された処理を確認することができます．

In [ ]:
clstdf.parameters

`clstdf`は時速100kmの速度フィルタリング，ループの削除，0.05km範囲の圧縮がなされたデータであることがわかります．

#### クリーニングしたデータに対して速度を求める

クリーニングしたデータの速度を求め，インタラクティブマップに表示してみましょう．
このデータが最終的に採用されるデータになります．

まずは速度を求めます．

In [ ]:
# clstdfをTrajectoryCollectionに変換する
collection = mpd.TrajectoryCollection(clstdf.to_geodataframe(), 'uid', t='datetime')
# TrajectoryCollectionの速度を計算する
collection.add_speed(overwrite=True, units=("km", "h"))
# TrajectoryCollectionを日付でソートする
collection.trajectories.sort(key=lambda x: x.get_start_time())


次にインタラクティブマップに表示します．背景地図として空中写真も選べるようにしました．

In [ ]:
mapcenter = [32.92, 131.1]

m = folium.Map(
    location=mapcenter,
    # tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    # attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
    #     # 地理院タイル</a>',
    # name='淡色地図',
    zoom_start=11
)

# 地理院タイル（淡色地図）を追加
# '全国最新写真（シームレス）'も面白いかも
# https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg
tile_layer = folium.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='地理院地図'
)
tile_layer.add_to(m)

# 地理院タイル（淡色地図）を追加
# '全国最新写真（シームレス）'も面白いかも
# https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg
tile_layer = folium.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='全国最新写真（シームレス）'
)
tile_layer.add_to(m)

# カスタムCSSを追加してタイルレイヤーをモノクロ化
m.get_root().html.add_child(folium.Element("""
<style>
    .leaflet-tile {
        filter: grayscale(100%);
    }
</style>
"""))

# TrajectoryCollectionの各Trajectoryをプロット
dt_fmt = '%Y-%m-%d'
for trajectory in collection.trajectories:
    start_time = trajectory.get_start_time().strftime(dt_fmt)
    trajectory.explore(
        m=m,
        column="speed",
        cmap='rainbow',
        vmin=0, vmax=100,
        legend=False,
        style_kwds={'weight': 5, 'opacity': 0.7},
        name=f'{start_time}'
    )


folium.LayerControl().add_to(m)
display(m)

## GPSデータからフローを作成する

クリーニングしたTrajectoryを使って，行動の発着地を結んだフローを作成します．

scikit-mobilityにはTrajDataFrameからフローを示すFlowDataFrameを作詞する機能があります．
主に立ち寄り地とTessellationの空間結合を行う`tdf.mapping`メソッドと，
`tdf.mapping`を行った後にTrajDataFrameをFlowDataFrameに変換する`tdf.to_flowdataframe`メソッドを使うのですが，
scikit-mobilityに対応しているGeoPandasのバージョンが古いため今回のプログラムでは動きません．
このため，自前でフローを作成します．

フローの作成手順は以下になります．

1. TrajDataFrameから立ち寄り地点を抽出する
1. 近接している立ち寄り地点をクラスタリングする
2. クラスタリングした立ち寄り地点から発着地の集計単位になるテッセレーションを作成する
3. テッセレーションの代表点を求める
4. 立ち寄り地点間のOD作成
5. ODからフローの作成

### 立ち寄り地の検出

クリーニングしたTrajDataFrame（`clstdf'）に対して`detection.stay_location`を用いて立ち寄り地点を検出します．
求めた立ち寄り地点はTrajDataFrameになります．

ここでは半径200m（0.2km）以内に5分間以上止まった場合に立ち寄りとします．
求めた立ち寄り地点のTrajDataFrameは`stops`とします．

In [ ]:
stops = detection.stay_locations(clstdf, minutes_for_a_stop=5, spatial_radius_km=0.2)

確認のため立ち寄り地点をプロットします．

In [ ]:
# 立ち寄り地をプロット
stops.plot_trajectory(zoom=12, weight=3, opacity=0.9, max_users=100, max_points=None)

#### 立ち寄り地のクラスタリング

`clustering.cluster`を使って同一地点と思われる立ち寄り地点をまとめます．
属性`cluster`の値は，0ほどクラスターメンバーが多い事を示します．
今回は半径1km以内を同一の立ち寄り地としました．
クラスタリング済みの立ち寄り地点は`cstops`とします．

In [ ]:
cstops = clustering.cluster(stops, cluster_radius_km=1, min_samples=1)

クラスタリングした立ち寄り地点をGeoDataFrameに変換し，インタラクティブマップに表示します．

In [ ]:
mapcenter = [32.92, 131.1]

m = folium.Map(
    location=mapcenter,
    # tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    # attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
    #     # 地理院タイル</a>',
    # name='淡色地図',
    zoom_start=11
)

# 地理院タイル（淡色地図）を追加
# '全国最新写真（シームレス）'も面白いかも
# https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg
tile_layer = folium.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='地理院地図'
)
tile_layer.add_to(m)

# カスタムCSSを追加してタイルレイヤーをモノクロ化
m.get_root().html.add_child(folium.Element("""
<style>
    .leaflet-tile {
        filter: grayscale(100%);
    }
</style>
"""))

cstops.to_geodataframe().explore(
    m=m,
    column='cluster',
    style_kwds={'weight': 5})

folium.LayerControl().add_to(m)
display(m)


#### メッシュ単位のフローデータの作成

立ち寄り地点を空間集計するためにテッセレーションを作成します．
テッセレーションは任意のポリゴンデータを使うことができますが，
scikit-mobilityの`tilers.tiler`を使って方形メッシュのテッセレーションを作成することもできます．

`tilers.tiler`はパラメーターとして範囲を示す`base_shape`を指定できます．
`base_shape`としてGeoDataFrameに変換した立ち寄り地点のデータ`cstops`を指定すれば，
立ち寄り地点の範囲のテッセレーションが作成できます．

ここでは`cstops`の範囲の5km正方メッシュのテッセレーションを作成します．

In [ ]:
# クラスタリングした立ち寄り地からテッセレーションを作成する

# クラスタリングした立ち寄り地をGeoDataFrameに変換
cstops_gdf = cstops.to_geodataframe()
# GeoDataFrameを元にテッセレーションを作成
tessellation = tilers.tiler.get("squared", base_shape=cstops_gdf, meters=5000)

作成したテッセレーションとクラスタリングした立ち寄り地点を地図表示します．

In [ ]:
mapcenter = [32.92, 131.1]

m = folium.Map(
    location=mapcenter,
    zoom_start=11
)

# 地理院タイル（淡色地図）を追加
# '全国最新写真（シームレス）'も面白いかも
# https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg
tile_layer = folium.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='地理院地図'
)
tile_layer.add_to(m)

# カスタムCSSを追加してタイルレイヤーをモノクロ化
m.get_root().html.add_child(folium.Element("""
<style>
    .leaflet-tile {
        filter: grayscale(100%);
    }
</style>
"""))

tessellation.explore(
    m=m,
    name='Tessellation',
    style_kwds={'fillColor': 'none'}
)

cstops_gdf.explore(
    m=m,
    column='cluster',
    name='Clustered Stops',
    style_kwds={'weight': 5}
)

folium.LayerControl().add_to(m)
display(m)

#### クラスタリングした立ち寄り地に (Origin) を設定する

テッセレーションにはメッシュのIDとなる`tile_id`があります．
この`tile_ID`をクラスタリングした立ち寄り地点に空間結合し，
どの立ち寄り地点がどのメッシュに含まれているか識別できるようにします．
この`tile_ID`が，ある立ち寄り地点の「発地ID（origin）」になります．

クラスタリングした立ち寄り地点とテッセレーションを`sjoin`メソッドで空間結合します．
この際，結合は左結合（`cstops`に合わせて結合）で，空間関係は「含まれる（`within`）」になります．
これで立ち寄り地点の属性にその立ち寄り地点を含むメッシュの属性が結合されます．
結合後の属性名`tile_ID`を`origin`に変更し，不要な列`index_right`を削除します．
最後に属性`origin`の型を実数にキャストし，着地IDを格納する属性`destination`を確保します．


In [ ]:
# 空間結合で発地を設定する．発地ID（originの値）はTessellationのtile_IDを使用する．
temp = cstops_gdf.to_crs(epsg=4326).sjoin(tessellation, how='left', predicate='within')\
  .rename(columns={'tile_ID': 'origin'})\
  .drop('index_right', axis=1)

# 発地（origin）の型をfloat64に変換
temp.astype({'origin': np.float64})
# 着地（destination）の列を追加
temp['destination'] = np.float64(0)


#### クラスタリングした立ち寄り地に着地 (Destination) を設定する

次に着地（destination）を設定します．
着地はある立ち寄り地点から次にどの着地点に移動するかを示したもので，
発地と同様にメッシュのIDで表します．

ここである日の立ち寄り地点の一部を見てみます．

In [ ]:
temp[temp['uid']==0].head(10)

最初の発地（`origin`）はメッシュ番号70です．
`datetime`はその立ち寄り地点に着いた時刻，`leaving_datetime`は立ち寄り地点を離れた時刻です．
データは時刻順に並んでいるため，ある立ち寄り地点の着地は次の立ち寄り地点の発地（`origin`）になります．

従って，時系列順に発地のリストを作成し，一つずらしたものをその日の立ち寄り地の着地に入れれば良いことになります．
最後の立ち寄り地点だけは着地が無いために削除します．

以下では上記を行い．最終的に発着地が加わった立ち寄り地点を`od`として保存します．


In [ ]:
# 発着地IDを含むGeoDataFrameを作成
od = gpd.GeoDataFrame()

# uidごとに処理するため，uidのユニーク値をリスト化する
uids = temp['uid'].unique().astype(int).tolist()
# uidごとに処理を行う
for uid in uids:
  # uidごとのデータをtempから抽出
  temp_origin = temp[temp['uid'] == uid]
  # 着地のリストを作成する．着地は発地を1つずらしたもの
  temp_dest = np.roll(temp_origin['origin'], -1).astype(np.float64)
  # 最後のデータは着地がないため，nanにする
  temp_dest[-1] = np.nan
  # 着地の列を追加する
  temp_origin.loc[:, 'destination'] = temp_dest
  # 結果をodに追加する
  od = pd.concat([od, temp_origin])
# 着地のない行（着地がnanになっている行）を削除する
od.dropna(inplace=True)
# originとdestinationの型をint64に変換する
od = od.astype({'origin': np.int64, 'destination': np.int64})

作成した`OD`を確認してみます．
ある立ち寄り地点の着地（`destination`）が次の地点の発地（'origin'）になっています．

In [ ]:
od.head(10)

発着地が揃った立ち寄り地点データが作成できたら，
発着地の組み合わせごとにカウントしして縦持ちのOD行列を作成します．

作成したOD行列は`odc`として保存します．

In [ ]:
# originとdestinationの組み合わせごとのカウントを取得する
odc = od.groupby(['origin', 'destination']).size().rename('count').reset_index()

ODCも確認してみます．

In [ ]:
odc.head(10)

ちゃんとカウントできているようです．

#### フローデータの作成

OD行列（`odc`）の`origin`から`destination`へラインを引いてフローを作成します．
`origin`と`destination`の座標は，テッセレーションの対応するメッシュの代表点とします．

テッセレーションの各メッシュの代表点の座標を求め,`cts_cent`に格納します．

In [ ]:
# Tessellationの代表点の座標を取得する
# 重心だと，Tessellationの形状によっては外に出てしまうことがあるため，
# 代表点（representative_point）を使用する
cts_cent = gpd.GeoDataFrame(tessellation.representative_point())\
  .rename(columns={0: 'geometry'})\
  .set_geometry('geometry')\
  .set_crs(epsg=4326)

確認します．

In [ ]:
cts_cent.head(10)

テッセレーションの代表点（`cts_cent`）とテッセレーション（`tesselation`）を空間結合し，
テッセレーションの代表点（`cts_cent`）にの`tile_ID`を加えます．

In [ ]:
# Tessellationの代表点にtile_IDを空間結合する
cts_id = cts_cent.sjoin(tessellation, how='left', predicate='within')\
  .astype({'tile_ID': np.int64})\
  .drop('index_right', axis=1)


確認します．

In [ ]:
cts_id.head(10)

ODの発地（`origin`）と着地（`destination`）に座標を結合し，
それぞれ`odc_o`と`odc_d`として保存します．

In [ ]:
# originの座標を結合
odc_o = gpd.GeoDataFrame(odc.merge(cts_id, left_on='origin', right_on='tile_ID', how='left'))
# destinationの座標を結合
odc_d = gpd.GeoDataFrame(odc.merge(cts_id, left_on='destination', right_on='tile_ID', how='left'))


結果を確認します．

In [ ]:
odc_o.head(10)

`odc_d`も確認します．

In [ ]:
odc_d.head(10)

`odc_o`から`odc_d`にラインを引き，フローを作成（`flow_gdf`）します．

In [ ]:
# flowを示すGeoDataFrameを作成する
flow_gdf = gpd.GeoDataFrame(columns=['geometry'], crs=odc_o.crs)
# originとdestinationの座標の組からflowを作成する
for ori, des in zip(odc_o.geometry, odc_d.geometry):
  # originとdestinationの座標からflowを作成
  segment = LineString([[ori.x, ori.y], [des.x, des.y]])
  # flowをGeoDataFrameに追加
  flow_gdf = pd.concat([flow_gdf, gpd.GeoDataFrame({'geometry': [segment]})])
# flowにcountを追加
flow_gdf['count'] = list(odc['count'])

作成したフローを表示してみましょう．

In [ ]:
flow_gdf.sort_values('count', ascending=True)\
  .plot(column='count', cmap='rainbow', legend=True, vmax=5)

インタラクティブマップでも表示してみます．

In [ ]:
mapcenter = [32.92, 131.1]

m = folium.Map(
    location=mapcenter,
    zoom_start=11
)

# 地理院タイル（淡色地図）を追加
# '全国最新写真（シームレス）'も面白いかも
# https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg
tile_layer = folium.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='地理院地図'
)
tile_layer.add_to(m)

# カスタムCSSを追加してタイルレイヤーをモノクロ化
m.get_root().html.add_child(folium.Element("""
<style>
    .leaflet-tile {
        filter: grayscale(100%);
    }
</style>
"""))

tessellation.explore(
    m=m,
    name='Tessellation',
    style_kwds={'fillColor': 'none'}
)

cstops_gdf.explore(
    m=m,
    column='cluster',
    name='Clustered Stops',
    style_kwds={'weight': 5}
)

flow_gdf.explore(
    m=m,
    column='count',
    style_kwds={'weight': 3},
    name='flow',
    vmax=5
)

folium.LayerControl().add_to(m)
display(m)

#### 結果をGeoJSONに保存する

立ち寄り地（`cstops_gdf`），テッセレーション（`tesselalation`），フロー（`flow_gdf`）をGeoJSONでファイル出力します．
すべてGeoDataFrameなので，`to_file`メソッドでファイル出力できます．
今回はGeoJSONで出力しましたが，GeoPackageやシェープファイルでの出力も可能です．
ただし，ファイル形式によってはデータの型が保持されないことがあるので注意が必要です．
例えばシェープファイルは時刻を保持する型がありません．

In [ ]:
# 立ち寄り地（cstops_gdf）をGeoJSONで保存する
cstops_gdf.to_file(base_dir + 'cstops.geojson', driver='GeoJSON')
# TessellationをGeoJSONで保存する
tessellation.to_file(base_dir + 'mesh_tessellation.geojson', driver='GeoJSON')
# フロー（flow_gdf)をGeoJSONで保存する
flow_gdf.to_file(base_dir + 'mesh_flow.geojson', driver='GeoJSON')

#### 小地域単位のフローデータの作成

正方メッシュのテッセレーションではなく，小地域をテッセレーションにした場合も実行してみましょう．
ファイル`r2ka.fgb`はFlatGeoBuf型式の小地域ポリゴンで，e-Statからダウンロードした熊本県と宮崎県のデータから作成しました．

立ち寄り地点は正方メッシュのテッセレーションのフロー作成の時に使ったものをそのまま利用します．
一気に実行するようになっているので，プログラムを読み取ってみてください．


In [ ]:
# 対象地域の小地域ポリゴンからtessellationを作成する

# 熊本県・宮崎県の小地域ポリゴンのparquetからデータを読み込み，
# 緯度経度をWGS84に変換してGeoDataFrameに格納する
tessellation = gpd.read_file(base_dir + 'r2ka.fgb')\
  .to_crs(epsg=4326)\
  .reset_index()

# 必要な列のみを残す
tessellation = tessellation[['KEY_CODE', 'PREF_NAME', 'CITY_NAME', 'S_NAME', 'geometry']]
# indexをtile_IDにする
tessellation['tile_ID'] = tessellation.index

# 空間結合で発地を設定する．発地ID（originの値）はTessellationのtile_IDを使用する．
temp = cstops_gdf.to_crs(epsg=4326).sjoin(tessellation, how='left', predicate='within')\
  .rename(columns={'tile_ID': 'origin'})\
  .drop('index_right', axis=1)

# 発地（origin）の型をfloat64に変換
temp.astype({'origin': np.float64})
# 着地（destination）の列を追加
temp['destination'] = np.float64(0)

# 発着地IDを含むGeoDataFrameを作成
od = gpd.GeoDataFrame()

# uidごとに処理するため，uidのユニーク値をリスト化する
uids = temp['uid'].unique().astype(int).tolist()
# uidごとに処理を行う
for uid in uids:
  # uidごとのデータをtempから抽出
  temp_origin = temp[temp['uid'] == uid]
  # 着地のリストを作成する．着地は発地を1つずらしたもの
  temp_dest = np.roll(temp_origin['origin'], -1).astype(np.float64)
  # 最後のデータは着地がないため，nanにする
  temp_dest[-1] = np.nan
  # 着地の列を追加する
  temp_origin.loc[:, 'destination'] = temp_dest
  # 結果をodに追加する
  od = pd.concat([od, temp_origin])
# 着地のない行（着地がnanになっている行）を削除する
od.dropna(inplace=True)
# originとdestinationの型をint64に変換する
od = od.astype({'origin': np.int64, 'destination': np.int64})

# originとdestinationの組み合わせごとのカウントを取得する
odc = od.groupby(['origin', 'destination']).size().rename('count').reset_index()

# Tessellationの代表点の座標を取得する
cts_cent = gpd.GeoDataFrame(tessellation.representative_point())\
  .rename(columns={0: 'geometry'})\
  .set_geometry('geometry')\
  .set_crs(epsg=4326)

# Tessellationの代表点にtile_IDを空間結合する
cts_id = cts_cent.sjoin(tessellation, how='left', predicate='within')\
  .astype({'tile_ID': np.int64})\
  .drop('index_right', axis=1)

# originの座標を結合
odc_o = gpd.GeoDataFrame(odc.merge(cts_id, left_on='origin', right_on='tile_ID', how='left'))
# destinationの座標を結合
odc_d = gpd.GeoDataFrame(odc.merge(cts_id, left_on='destination', right_on='tile_ID', how='left'))

# flowを示すGeoDataFrameを作成する
flow_gdf = gpd.GeoDataFrame(columns=['geometry'], crs=odc_o.crs)
# originとdestinationの座標の組からflowを作成する
for ori, des in zip(odc_o.geometry, odc_d.geometry):
  # originとdestinationの座標からflowを作成
  segment = LineString([[ori.x, ori.y], [des.x, des.y]])
  # flowをGeoDataFrameに追加
  flow_gdf = pd.concat([flow_gdf, gpd.GeoDataFrame({'geometry': [segment]})])
# flowにcountを追加
flow_gdf['count'] = list(odc['count'])

とりあえずプロットして結果を確認してみましょう．

In [ ]:
flow_gdf.sort_values('count', ascending=True)\
  .plot(column='count', cmap='rainbow', legend=True, vmax=5)

インタラクティブマップでも表示してみます．

In [ ]:
mapcenter = [32.92, 131.1]

m = folium.Map(
    location=mapcenter,
    zoom_start=11
)

# 地理院タイル（淡色地図）を追加
# '全国最新写真（シームレス）'も面白いかも
# https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg
tile_layer = folium.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='地理院地図'
)
tile_layer.add_to(m)

# カスタムCSSを追加してタイルレイヤーをモノクロ化
m.get_root().html.add_child(folium.Element("""
<style>
    .leaflet-tile {
        filter: grayscale(100%);
    }
</style>
"""))

tessellation.explore(
    m=m,
    name='Tessellation',
    style_kwds={'fillColor': 'none'}
)

cstops_gdf.explore(
    m=m,
    column='cluster',
    name='Clustered Stops',
    style_kwds={'weight': 5}
)

flow_gdf.sort_values('count', ascending=True).explore(
    m=m,
    column='count',
    cmap='rainbow',
    style_kwds={'weight': 3, 'opacity': 0.7},
    name='flow',
    vmax=5
)

folium.LayerControl().add_to(m)
display(m)

##### 立ち寄り地，Tessellation，flowlineをGeoJSONで保存する

こちらの結果もGeoJSONでファイル保存します．

In [ ]:
# 立ち寄り地（cstops_gdf）をGeoJSONで保存する
cstops_gdf.to_file(base_dir + 'cstops.geojson', driver='GeoJSON')
# TessellationをGeoJSONで保存する
tessellation.to_file(base_dir + 'area_tessellation.geojson', driver='GeoJSON')
# フロー（flow_gdf)をGeoJSONで保存する
flow_gdf.to_file(base_dir + 'area_flow.geojson', driver='GeoJSON')


# MASデータの分析


MASはマルチエージェントシミュレーションの略で，
個々に行動ルールを持ち互いに影響し合う多数のエージェントが行動することにより，
全体としてどのような振る舞いが生じるかをシミュレートするものです．

この章では，熊本県阿蘇市内牧地区を対象に行ったMASの結果を用いて，
データのクリーニングおよび縮約を行い，
行動のボトルネックとなる場所を抽出することを目的とします．

MASの結果をアニメーション化したものは，以下のリンクからご覧下さい．

[講習用動画（YouTube）](https://youtu.be/-x3bb2BwD7k)

使用したMASプログラムについては，
[GIS Day in 東京 2023 Eコースマルチエージェントシミュレーションを使った人の動きのシミュレーション」](https://github.com/bokutachi256/gisday2023)をご覧下さい．

## ファイルの読み込み

MASの結果はFlatGeoBuf型式で保存されています．
このファイルを読み込んでGeoDataFrameにします．

In [ ]:
# 読み込むファイル名
mas_file = 'gisday2024_agent_vars.fgb'
# ファイルの読み込み
indata = gpd.read_file(base_dir + mas_file).to_crs(epsg=4326)
# ゴール後のワープ先を削除
mas_data = indata[indata['Goal'] == 0]
# スタート前の滞留を削除
mas_data = mas_data[mas_data['start_move']==0]

読み込んだデータを確認します．

In [ ]:
len(mas_data)

読み込んだデータをTrajDataFrameに変換します．

In [ ]:
df = pd.DataFrame()
df['lon'] = mas_data.geometry.x
df['lat'] = mas_data.geometry.y
df['datetime'] = mas_data['Time']
df['id'] = mas_data['AgentID']
df['evac_no'] = mas_data['Evac_no']

tdf = skmob.TrajDataFrame(
  df,
  latitude='lat',
  longitude = 'lon',
  datetime='datetime',
  user_id='id'
)


Trajectoryを表示します．

In [ ]:
tdf.plot_trajectory(zoom=12, weight=3, opacity=0.9, max_users=1000, start_end_markers=False)

## 経路データのクリーニング

時速100km以上とループを削除します．

In [ ]:
ftdf = filtering.filter(tdf, max_speed_kmh=100, include_loops=True)

In [ ]:
print(f'オリジナル: {len(tdf)}')
print(f'ループを含む速度フィルタリング後: {len(ftdf)}')

プロットして確認します．

In [ ]:
ftdf.plot_trajectory(zoom=12, weight=3, opacity=0.9, max_users=1000, start_end_markers=False)

## 経路データからの滞留ポイント抽出

半径20m以内に2分以上滞留していたものを検出します．

In [ ]:
stdf = detection.stay_locations(
  ftdf, stop_radius_factor=0.5, minutes_for_a_stop=2.0,
  spatial_radius_km=0.02, leaving_time=True
)

結果を確認します．

In [ ]:
stdf.head(10)

滞留ポイント数も確認します．

In [ ]:
print(f'滞留ポイント数: {len(stdf)}')

GeoDataFrameに変換して滞留ポイントをインタラクティブマップに表示します．

In [ ]:
stay_gdf = stdf.to_geodataframe()

mapcenter = [32.975, 131.04]

m = folium.Map(
  location=mapcenter,
  tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
  attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">地理院タイル</a>',
  name='淡色地図',
  zoom_start=15
)

stay_gdf.to_crs(epsg=3857).explore(
  m=m,
  color = 'red',
  style_kwds={'weight': 5}
)

速度を計算するため
TrajDataFrameをMovingDataFrameのTrajectoryCollectionに変換します．

In [ ]:
ftdf_gdf = ftdf.to_geodataframe()
collection = mpd.TrajectoryCollection(ftdf_gdf, 'uid', t='datetime')

TrajectoryCollectionに対して速度を求めます．

In [ ]:
collection.add_speed(overwrite=True, units=("km", "h"))

すべてのtrajectoryの速度をプロットすると大事になるから最初の一つだけプロットします．

In [ ]:
collection.trajectories[0].plot(column="speed", legend=True)

## 経路のスムージングと集計

MovingPandasではデータの間引きではなく関数で補間することによる経路のスムージングができます．
試しに経路の一つにパラメーターを変えてスムージングを行い，
結果を比較してみます．

In [ ]:
# デフォルトの値でスムージングしてみる
smooth_01 = mpd.KalmanSmootherCV(collection.trajectories[12]).smooth(
    process_noise_std=0.5,
    measurement_noise_std=1
)
# スムージングしてみる
smooth_02 = mpd.KalmanSmootherCV(collection.trajectories[12]).smooth(
    process_noise_std=10,
    measurement_noise_std=1
)
# スムージングしてみる
smooth_03 = mpd.KalmanSmootherCV(collection.trajectories[12]).smooth(
    process_noise_std=0.5,
    measurement_noise_std=50
)
# スムージングしてみる
smooth_04 = mpd.KalmanSmootherCV(collection.trajectories[12]).smooth(
    process_noise_std=10,
    measurement_noise_std=50
)

スムージング前のデータとスムージング後のデータの速度をプロットして比較します．

In [ ]:
mapcenter = [32.92, 131.1]

m = folium.Map(
    location=mapcenter,
    zoom_start=11
)

# 地理院タイル（淡色地図）を追加
# '全国最新写真（シームレス）'も面白いかも
# https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg
tile_layer = folium.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='地理院地図'
)
tile_layer.add_to(m)

# カスタムCSSを追加してタイルレイヤーをモノクロ化
m.get_root().html.add_child(folium.Element("""
<style>
    .leaflet-tile {
        filter: grayscale(100%);
    }
</style>
"""))
collection.trajectories[12].explore(
    m=m,
    cmap='rainbow',
    style_kwds={'weight': 5, 'opacity': 0.7},
    column='speed',
    name='original'
)

smooth_01.explore(
    m=m,
    cmap='rainbow',
    style_kwds={'weight': 5, 'opacity': 0.7},
    column='speed',
    name='sm01, pn=0.5, mn=1',
    legend=False
)

smooth_02.explore(
    m=m,
    cmap='rainbow',
    style_kwds={'weight': 5, 'opacity': 0.7},
    column='speed',
    name='sm02, pn=10, mn=1',
    legend=False
)

smooth_03.explore(
    m=m,
    cmap='rainbow',
    style_kwds={'weight': 5, 'opacity': 0.7},
    column='speed',
    name='sm03, pn=0.5, mn=50',
    legend=False
)

smooth_04.explore(
    m=m,
    cmap='rainbow',
    style_kwds={'weight': 5, 'opacity': 0.7},
    column='speed',
    name='sm04, pn=10, mn=50',
    legend=False
)

folium.LayerControl().add_to(m)
display(m)

データ全体をスムージングすると18分くらいかかるため，ここでは全体へのスムージングは行わない．行う場合は以下を実行する．

```
# process_noise=0.5，measurement_noise=50でスムージングする
smooth = mpd.KalmanSmootherCV(generalized_gdf).smooth(
    process_noise_std=0.5,
    measurement_noise_std=50
)
```

スムージングの代わりにgeneralizerでデータを間引いてみます．
MinTimeDeltaGeneralizerを使うと時間で間引くことができます．
ここでは30秒間隔で間引きます．

In [ ]:
generalized = mpd.MinTimeDeltaGeneralizer(collection).generalize(tolerance=timedelta(seconds=30))

時間で間引いた経路の一部を地図表示してみる．

In [ ]:
mapcenter = [32.92, 131.1]

m = folium.Map(
    location=mapcenter,
    zoom_start=11
)

# 地理院タイル（淡色地図）を追加
# '全国最新写真（シームレス）'も面白いかも
# https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg
tile_layer = folium.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='地理院地図'
)
tile_layer.add_to(m)

# カスタムCSSを追加してタイルレイヤーをモノクロ化
m.get_root().html.add_child(folium.Element("""
<style>
    .leaflet-tile {
        filter: grayscale(100%);
    }
</style>
"""))

collection.trajectories[12].explore(
    m=m,
    cmap='rainbow',
    style_kwds={'weight': 5, 'opacity': 0.7},
    column='speed',
    name='original'
)

generalized.trajectories[12].explore(
    m=m,
    cmap='rainbow',
    style_kwds={'weight': 5, 'opacity': 0.7},
    column='speed',
    name='generalized',
    legend=False
)

folium.LayerControl().add_to(m)
display(m)

時間で間引いたデータをスムージングしてみましょう．

In [ ]:
# process_noise=0.5，measurement_noise=50でスムージングする
generalized_smooth = mpd.KalmanSmootherCV(generalized).smooth(
    process_noise_std=0.5,
    measurement_noise_std=50
)

間引いてスムージングした経路をマップ表示してみます．

In [ ]:
mapcenter = [32.92, 131.1]

m = folium.Map(
    location=mapcenter,
    zoom_start=11
)

# 地理院タイル（淡色地図）を追加
# '全国最新写真（シームレス）'も面白いかも
# https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg
tile_layer = folium.TileLayer(
    tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">\
        地理院タイル</a>',
    name='地理院地図'
)
tile_layer.add_to(m)

# カスタムCSSを追加してタイルレイヤーをモノクロ化
m.get_root().html.add_child(folium.Element("""
<style>
    .leaflet-tile {
        filter: grayscale(100%);
    }
</style>
"""))

collection.trajectories[12].explore(
    m=m,
    cmap='rainbow',
    style_kwds={'weight': 5, 'opacity': 0.7},
    column='speed',
    name='original'
)

generalized.trajectories[12].explore(
    m=m,
    cmap='rainbow',
    style_kwds={'weight': 5, 'opacity': 0.7},
    column='speed',
    name='generalized',
    legend=False
)

generalized_smooth.trajectories[12].explore(
    m=m,
    cmap='rainbow',
    style_kwds={'weight': 5, 'opacity': 0.7},
    column='speed',
    name='smooth',
    legend=False
)

folium.LayerControl().add_to(m)
display(m)

## 計算結果を出力する

generalizeした経路とsmoothした経路をGeoDataFrameに変換し，
滞留ポイントと合わせて保存します．
これらのデータは一つのGeoPackageに別レイヤーとして保存します．

In [ ]:
# GeoDataFrameに変換する
generalized_gdf = generalized.to_line_gdf()
generalized_smooth_gdf = generalized_smooth.to_line_gdf()

# GeoDataFrameをGeoPackageとして保存する
generalized_gdf.to_file(base_dir + 'MAS_analysis.gpkg', layer='generalized', driver='GPKG')
generalized_smooth_gdf.to_file(base_dir + 'MAS_analysis.gpkg', layer='generalized_smooth', driver='GPKG')
stay_gdf.to_file(base_dir + 'MAS_analysis.gpkg', layer='stay', driver='GPKG')